<a href="https://colab.research.google.com/github/r1marcus/TraintheTrainer-Daten/blob/main/01_Daten_load_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modul Daten

Dieses Jupyter Notebook soll einen Überblick über die Aufbereitung, Bereinigung und Visualisierung von Daten vermitteln. 

Für diesen Zweck wurde die Datenbank Gas Sensor Array Drift verwendet, heruntergeladen und vorher für diesen Zweck manipuliert (Gas- und Konzentrationsklassenlaben wurden zu einem Label zusammen geführt und dabei drei verschiedene Konzentrationsstufen definiert: low, medium und high). <br>
Der vollständige Datensatz kann auf <br>

http://archive.ics.uci.edu/ml/datasets/Gas+Sensor+Array+Drift+Dataset+at+Different+Concentrations

heruntergeladen werden. Außerdem werden verschiedene Informationen zu den Daten bereitgestellt. <br>

Der Datensatz enthält 13.910 Messungen von 16 chemischen Sensoren, welche abwechselnd sechs verschiedene Gase (Ethanol, Ethen, Ammoniak, Acetaldehyd, Aceton und Toluol) in verschiedenen Konzentrationen untersuchen. <br>
Der Datensatz wurde zwischen Januar 2008 und Februar 2011 (36 Monaten) erhoben. <br>
Jede Messung liefert anhand der 16 vorhandenen Sensoren eine 16-Kanal-Zeitreihe. <br>
Es werden zwei Hauptfeatures in diesem Datensatz betrachtet: <br>
$\rightarrow~$ (i) das stationäre Feature (steady-state) bezeichnet als `DR`, definiert als die maximale Widerstandsänderung in Bezug auf eine Basis, sowie die normalisierte Version davon (`|DR|`). <br>
$\rightarrow~$ (ii) ein Ansammlung an Features welche die Sensordynamik der gesamten Messung wiederspiegelt (`EMAi` und `EMAd` für verschiedene $\alpha$-Werte). <br>

Im folgendem Beispiel werden wir uns nur mit dem `DR` Feature auseinandersetzen. Da jeder der 16 Sensoren diesen Messwert liefert, sind auch 16 verschiedene Messwerte von `DR` vorhanden (`DR_1` des ersten Sensors, `DR_2`, des zweiten Sensorns und wo weiter). 

## Laden des Pandas DataFrames

Als erstes müssen alle Beispieldatensätze geladen werden. Dieser liegen in 10 verschiedene csv Dateien vor (batch0.csv bis batch9.csv). Anschließend werden diese zu einem gemeinsamen Datensatz zusammengeführt.

In [1]:
import scripts.load

scripts.load.load()

Download dataset from URL: http://archive.ics.uci.edu/ml/machine-learning-databases/00270/
Extract files to : /content/input/dataset_dat//driftdataset.zip
Convert dat files to csv
445 lines have been written for batch 1
1244 lines have been written for batch 2
1586 lines have been written for batch 3
161 lines have been written for batch 4
197 lines have been written for batch 5
2300 lines have been written for batch 6
3613 lines have been written for batch 7
294 lines have been written for batch 8
470 lines have been written for batch 9
3600 lines have been written for batch 10
13910 lines have been written in total
call load_svmlight_file for batch1.csv
call load_svmlight_file for batch2.csv
call load_svmlight_file for batch3.csv
call load_svmlight_file for batch4.csv
call load_svmlight_file for batch5.csv
call load_svmlight_file for batch6.csv
call load_svmlight_file for batch7.csv
call load_svmlight_file for batch8.csv
call load_svmlight_file for batch9.csv
call load_svmlight_file 

(                 target           DR_1  ...  EMAd0.01_16  EMAd0.1_16
 0           Ethanol low   20393.568401  ...    -0.975534   -1.706652
 1           Ethanol low   24232.596125  ...    -0.840539   -2.339802
 2           Ethanol low   35382.379620  ...    -2.317757   -2.811960
 3           Ethanol low   19577.647074  ...    -1.906482   -2.521623
 4           Ethanol low   58868.763765  ...    -3.470292   -4.943211
 ...                 ...            ...  ...          ...         ...
 13905        Aceton low   27906.236493  ...    -0.567691   -3.712709
 13906  Acetaldehyd high   38624.734479  ...    -6.832563  -13.906354
 13907       Aceton high  176798.949722  ...   -26.965712  -58.237848
 13908  Acetaldehyd high   26248.556196  ...    -3.988595  -15.729976
 13909       Aceton high  226931.931248  ...   -53.955260 -128.025950
 
 [13910 rows x 129 columns],
                  target         DR_1  ...  EMAd0.01_16  EMAd0.1_16
 0           Ethanol low   15596.1621  ...    -0.902241   -2.

In [2]:
df_modified, df_original,all_files_mod,all_files_original = scripts.load.load()


Download dataset from URL: http://archive.ics.uci.edu/ml/machine-learning-databases/00270/
Extract files to : /content/input/dataset_dat//driftdataset.zip
Convert dat files to csv
445 lines have been written for batch 1
1244 lines have been written for batch 2
1586 lines have been written for batch 3
161 lines have been written for batch 4
197 lines have been written for batch 5
2300 lines have been written for batch 6
3613 lines have been written for batch 7
294 lines have been written for batch 8
470 lines have been written for batch 9
3600 lines have been written for batch 10
13910 lines have been written in total
call load_svmlight_file for batch1.csv
call load_svmlight_file for batch2.csv
call load_svmlight_file for batch3.csv
call load_svmlight_file for batch4.csv
call load_svmlight_file for batch5.csv
call load_svmlight_file for batch6.csv
call load_svmlight_file for batch7.csv
call load_svmlight_file for batch8.csv
call load_svmlight_file for batch9.csv
call load_svmlight_file 

## Ein kurzer Blick auf die Datenstruktur

Um ein Gefühl für die Datenstruktur zu erhalten bietet Pandas die folgenden Methoden an: <br>

`head(n)` $\longrightarrow~$ Gibt die $n$ ersten Zeilen (default $n$ = 5) aus. Nützlich um schnell zu testen, ob das Objekt im richtigen Datentyp vorliegt. <br>
`info()` $\longrightarrow~$ Gibt eine übersichtliche Zusammenfassung des DataFrames aus, einschließlich des Index dtypes, des Spalten dtypes, Nicht-Null-Werte sowie den Speicherverbrauch. <br>
`describe()` $\longrightarrow~$ Erstellt deskriptive Statistiken, welche die zentrale Tendenz, Verteilung und Form der Verteilung eines Datensatzes unter Ausschluss von NaN-Werten zusammenfasst. <br>
`value_count()` $\longrightarrow~$ Gibt die Anzahl eindeutiger Werte in absteigender Reihenfolge wieder. Muss an einem Feature aufgerufen werden.

Werfen wir als erstes ein Blick auf die `head()` Methode:

In [3]:
df_modified.head()

,target,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1,EMAi0.1_1,EMAd0.001_1,EMAd0.01_1,EMAd0.1_1,DR_2,|DR|_2,EMAi0.001_2,EMAi0.01_2,EMAi0.1_2,EMAd0.001_2,EMAd0.01_2,EMAd0.1_2,DR_3,|DR|_3,EMAi0.001_3,EMAi0.01_3,EMAi0.1_3,EMAd0.001_3,EMAd0.01_3,EMAd0.1_3,DR_4,|DR|_4,EMAi0.001_4,EMAi0.01_4,EMAi0.1_4,EMAd0.001_4,EMAd0.01_4,EMAd0.1_4,DR_5,|DR|_5,EMAi0.001_5,EMAi0.01_5,EMAi0.1_5,EMAd0.001_5,EMAd0.01_5,...,DR_12,|DR|_12,EMAi0.001_12,EMAi0.01_12,EMAi0.1_12,EMAd0.001_12,EMAd0.01_12,EMAd0.1_12,DR_13,|DR|_13,EMAi0.001_13,EMAi0.01_13,EMAi0.1_13,EMAd0.001_13,EMAd0.01_13,EMAd0.1_13,DR_14,|DR|_14,EMAi0.001_14,EMAi0.01_14,EMAi0.1_14,EMAd0.001_14,EMAd0.01_14,EMAd0.1_14,DR_15,|DR|_15,EMAi0.001_15,EMAi0.01_15,EMAi0.1_15,EMAd0.001_15,EMAd0.01_15,EMAd0.1_15,DR_16,|DR|_16,EMAi0.001_16,EMAi0.01_16,EMAi0.1_16,EMAd0.001_16,EMAd0.01_16,EMAd0.1_16
0,Ethanol low,55.755193,2.424440,1.557238,-2.407588,10.260477,-6.976443,-0.850537,12.358386,14877.177303,2.291253,1.903296,0.065235,9.003578,0.276662,-2.189309,-0.056127,2243.812546,2.773738,0.576261,0.949011,1.855935,-0.289560,-1.158054,-1.538703,2842.224094,2.661311,0.379207,0.944224,2.501789,-0.467707,-0.829613,-1.856729,590.172968,1.417311,0.219961,0.216369,0.423099,-0.258095,-0.342447,...,244.555432,3.488915,0.057342,0.529043,1.812166,-0.314740,0.103476,-1.107953,1268.757640,1.455465,0.103555,0.010852,1.536840,-0.336527,-0.425640,-0.998669,547.308286,2.114296,0.269631,0.384086,0.894435,-0.258995,-0.372599,-0.977316,2181.893314,2.927735,0.559219,-0.055755,0.840144,-1.946954,-1.811118,-3.638372,4278.770197,4.328248,0.170843,1.128315,0.230627,-0.681816,-1.853012,-2.421386
1,Ethanol low,37738.416343,2.226076,2.468068,8.622448,4.825957,-2.188393,-2.284583,4.986919,31960.914868,2.288001,5.715006,8.036136,7.710762,-4.103006,-4.006841,-6.276902,2709.769954,3.733252,0.806540,0.629598,2.009868,-0.938485,-1.023805,-2.944126,3873.826274,2.331920,1.378561,1.928038,1.713455,-0.451261,-0.690051,-1.321474,1397.443203,2.203160,0.270758,0.278714,0.693646,-0.412159,-0.485868,...,2774.850414,2.953224,0.813747,-0.109639,0.943185,-0.643989,-0.561567,-0.490602,1476.332092,2.116405,0.300252,0.419192,0.904244,-0.380007,-0.558497,-0.972898,1911.184291,2.181682,0.198536,0.970993,1.476231,-0.381137,-0.685067,-1.000074,6720.005038,3.797946,0.239322,2.172392,1.870746,-1.680633,-1.908373,-3.124882,3950.637525,4.048859,0.851745,0.981849,0.592716,-0.730532,-0.268224,-2.803655
2,Ethanol low,72973.350441,3.868816,6.701240,13.203769,6.580611,-2.645822,-3.961910,-30.809946,32599.384257,3.471114,7.129527,7.195569,8.036725,-5.178069,-10.674547,-4.865427,5531.090617,4.807085,1.293019,1.319274,1.726672,-1.192809,-1.346812,-2.967349,5007.287420,3.192944,1.003420,1.986011,1.187909,-0.855806,-1.009062,-1.488021,1274.629795,2.204765,0.347758,0.489493,1.049796,-0.390270,-0.561567,...,4680.902012,5.293913,1.268856,0.920701,1.089302,-0.848045,-1.441053,-0.638931,1407.327371,2.267968,0.437177,0.783689,1.944902,-0.415473,-0.718643,-1.278990,2002.690477,2.871872,0.509719,1.192582,1.426738,-0.522235,-0.843971,-1.028356,5860.778313,5.076898,1.945874,3.986142,3.236788,-1.368505,-2.699548,-3.344537,6717.974802,5.015145,1.481470,2.249118,1.566714,-0.644967,-1.891913,-3.901128
3,Ethanol low,58697.868903,3.034353,15.222149,17.394803,36.171292,-6.812234,-10.720327,-29.405039,35932.420835,2.766051,11.122138,17.816157,35.029076,-7.124635,-8.664233,-25.611348,4769.355081,3.054158,1.527892,3.116626,2.483913,-0.972101,-1.750573,-2.261867,6636.241440,4.343514,1.346428,1.976212,3.217553,-1.412663,-2.013807,-1.475437,1686.638916,2.296729,0.434615,0.988241,1.146044,-0.316638,-0.518801,...,4956.553626,4.324549,1.460223,1.349313,3.054735,-0.995557,-0.998455,-2.538044,1713.027982,2.268103,0.735226,1.421336,2.091407,-0.441930,-0.632736,-1.328990,2054.417422,2.708843,0.781648,1.350912,1.778628,-0.422859,-0.776946,-1.047801,7352.541669,6.574527,1.976136,2.660222,6.821103,-1.993861,-1.855836,-3.411296,6895.013017,5.942690,2.470903,3.779397,4.534471,-0.523077,-2.024138,-3.460815
4,Ethanol l

Die erste Spalte verät uns das Klassenlabel. Die erste Instanz ist demnach eine Messung am Gas Ethanol bei niedriger Konzentration. Alle weitere Spalten beinhalten Daten die zu diesem Zeitpunkt bzw. Messpunkt erhoben worden sind. <br>

Rufen wir die `info()` Methode an den ersten fünf Spalten auf.

In [4]:
df_modified.iloc[:,:5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13910 entries, 0 to 13909
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   target       13910 non-null  object 
 1   DR_1         13895 non-null  float64
 2   |DR|_1       13897 non-null  float64
 3   EMAi0.001_1  13897 non-null  float64
 4   EMAi0.01_1   13888 non-null  float64
dtypes: float64(4), object(1)
memory usage: 543.5+ KB


Für Pandas Indexing und Slicing siehe https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html. <br>

`info()` listet uns die verschiedene dtypes auf: vier float64 und eins vom Typ object. Anhand der unterschiedlichen Instanzen (total: 13910, DR_1: 13892, ...) erkennen wir, dass der Datensatz unvollständig ist (NaNs können ein Grund dafür sein). <br>

Werfen wir als nächstes ein Blick auf die `describe()` Methode.

In [5]:
df_modified.iloc[:,:5].describe()

,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1
count,13895.000000,13897.000000,13897.000000,13888.000000
mean,50479.207790,6.640639,12.934288,18.737421
std,70133.325622,13.527197,17.669584,25.001936
min,-32840.457756,-9.597396,-7.290928,-14.132063
25%,7102.751046,2.287318,1.761861,2.595893
50%,20867.009930,3.995125,5.267600,7.760617
75%,63395.986787,8.633440,17.358825,25.911588
max,697084.442985,1335.971254,168.820369,229.082505


`describe()` bietet zusätzliche Informationen zu den jeweiligen Features. Dazu gehört die Anzahl, der Mittelwert (mean), die Standardabweichung (std) sowie das minimale, 25%, 50%, 75% und das maximale Quantil.

In [6]:
df_modified['target'].value_counts()

Aceton high           1834
Toluol low            1425
Ethen low             1190
Ammoniak high         1156
Ethanol low           1083
Ethen high            1082
Ethanol high           963
Acetaldehyd high       836
Ethen medium           654
Aceton low             642
Acetaldehyd low        565
Acetaldehyd medium     535
Aceton medium          533
Ethanol medium         519
Toluol medium          397
Ammoniak medium        311
Ammoniak low           174
Toluol high             11
Name: target, dtype: int64

Wir haben `value_counts()` an der `target` Spalte aufgerufen und sehen das Aceton medium mit 1924 Einträge über die meisten Messungen verfügt. Gefolgt von Toluol low und so weiter.

### Pandas Profiling

Im folgenden soll noch Pandas Profiling vorgestellt werden. Pandas Profiling bietet ähnlich wie die `describe()` Methode statistische Auswertungen. Mehr dazu kann auf https://github.com/pandas-profiling/pandas-profiling gelesen werden. <br>

Der folgende Code (auskommentiiert da dieser Schritt zeitintensiv ist) erstellt ein vollständigen Bericht und speichert diesen als `report.html` im lokalen Verzeichnis ab.

In [7]:
#profile = df_modified.profile_report(title='Pandas Profiling Report')
#profile.to_file(output_file="report.html")